In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('Courses.db')
cursor = conn.cursor()

1. students


| Column      | Type | Description                 |
| ----------- | ---- | --------------------------- |
| student\_id | TEXT | Primary key for the student |
| name        | TEXT | Student's name              |
| major       | TEXT | Student's major             |

2. courses

| Column     | Type    | Description                |
| ---------- | ------- | -------------------------- |
| course\_id | TEXT    | Primary key for the course |
| title      | TEXT    | Course title               |
| credits    | INTEGER | Number of credit hours     |

3. enrollments

(This is the junction table: represents which student enrolled in which course)

| Column      | Type | Description                               |
| ----------- | ---- | ----------------------------------------- |
| student\_id | TEXT | Foreign key referencing the student       |
| course\_id  | TEXT | Foreign key referencing the course        |
| grade       | TEXT | Student's grade in the course (A, B, ...) |

 This table represents a Many-to-Many relationship between students and courses.


In [3]:
cursor.execute("""
create table if not exists students(
  student_id text primary key,
  name text,
  major text
)
""")

cursor.execute("""
create table if not exists courses(
  course_id text primary key,
  title text,
  credits integer
)
""")

cursor.execute("""
create table if not exists enrollments(
  student_id text,
  course_id text,
  grade text,
  primary key (student_id,course_id),
  foreign key(student_id) references students(student_id),
  foreign key(course_id) references courses(course_id)
)
""")
conn.commit()

In [4]:
students_data = [
    ('S1', 'Lina', 'Computer Science'),
    ('S2', 'Omar', 'Data Science'),
    ('S3', 'Sara', 'Business'),
    ('S4', 'Yousef', 'Computer Science')
]

courses_data = [
    ('C1', 'Databases', 3),
    ('C2', 'Python Programming', 4),
    ('C3', 'Statistics', 3),
    ('C4', 'Machine Learning', 4)
]

enrollments_data = [
    ('S1', 'C1', 'A'),
    ('S1', 'C2', 'B'),
    ('S2', 'C2', 'A'),
    ('S2', 'C3', 'B'),
    ('S3', 'C3', 'A'),
    ('S3', 'C4', 'C'),
    ('S4', 'C1', 'A'),
    ('S4', 'C4', 'B')
]

cursor.executemany("INSERT INTO students VALUES (?, ?, ?)", students_data)
cursor.executemany("INSERT INTO courses VALUES (?, ?, ?)", courses_data)
cursor.executemany("INSERT INTO enrollments VALUES (?, ?, ?)", enrollments_data)
conn.commit()


In [5]:
df = pd.read_sql_query('select * from students',conn)
df

,student_id,name,major
0,S1,Lina,Computer Science
1,S2,Omar,Data Science
2,S3,Sara,Business
3,S4,Yousef,Computer Science


In [6]:
df = pd.read_sql_query('select * from courses',conn)
df

,course_id,title,credits
0,C1,Databases,3
1,C2,Python Programming,4
2,C3,Statistics,3
3,C4,Machine Learning,4


In [7]:
df = pd.read_sql_query('select * from enrollments',conn)
df

,student_id,course_id,grade
0,S1,C1,A
1,S1,C2,B
2,S2,C2,A
3,S2,C3,B
4,S3,C3,A
5,S3,C4,C
6,S4,C1,A
7,S4,C4,B


# Understanding Table Relationships (1:N and M:N)

🔸 One-to-Many (1:N) Relationship
Definition: One record in table A is related to many records in table B.

Example: One course can have many students enrolled in it, but each enrollment belongs to only one course.

🔹 Example:

Table courses:
course_id, title, credits

Table enrollments:
student_id, course_id, grade

One course → many enrollments.

🔸 Many-to-Many (M:N) Relationship
Definition: Many records in table A can be related to many records in table B.

Example:
Students can enroll in multiple courses, and each course can have multiple students.



---

# JOINs
# Using `INNER JOIN`
An INNER JOIN returns only the rows where there is a match in both tables.

In [ ]:
# Show all students and the courses they are enrolled in, along with their grades:
query = """
SELECT students.name, courses.title, enrollments.grade
FROM enrollments
INNER JOIN students ON enrollments.student_id = students.student_id
INNER JOIN courses ON enrollments.course_id = courses.course_id;
"""

df = pd.read_sql_query(query, conn)
df


,name,title,grade
0,Lina,Databases,A
1,Lina,Python Programming,B
2,Omar,Python Programming,A
3,Omar,Statistics,B
4,Sara,Statistics,A
5,Sara,Machine Learning,C
6,Yousef,Databases,A
7,Yousef,Machine Learning,B


# Using `LEFT JOIN`
A LEFT JOIN returns all rows from the left table, and the matching rows from the right table. If there’s no match, it fills with NULL.

In [8]:
# Show all students, and the courses they are enrolled in (if any):
# Show all students and the courses they are enrolled in, along with their grades:
query = """
SELECT students.name, courses.title, enrollments.grade
FROM students
LEFT JOIN enrollments ON students.student_id = enrollments.student_id
LEFT JOIN courses ON enrollments.course_id = courses.course_id;
"""

df = pd.read_sql_query(query, conn)
df


,name,title,grade
0,Lina,Databases,A
1,Lina,Python Programming,B
2,Omar,Python Programming,A
3,Omar,Statistics,B
4,Sara,Statistics,A
5,Sara,Machine Learning,C
6,Yousef,Databases,A
7,Yousef,Machine Learning,B




---


| Join Type    | Result                                                                           |
| ------------ | -------------------------------------------------------------------------------- |
| `INNER JOIN` | Only students who have actual enrollments in courses                             |
| `LEFT JOIN`  | All students, even those without any enrollments, with `NULL` for missing values |


---



In [9]:
cursor.execute("INSERT INTO students VALUES (?, ?, ?)", ('S5', 'Khaled', 'Artificial Intelligence'))
conn.commit()


In [10]:
# Show all students, and the courses they are enrolled in (if any):
# Show all students and the courses they are enrolled in, along with their grades:
query = """
SELECT students.name, courses.title, enrollments.grade
FROM students
LEFT JOIN enrollments ON students.student_id = enrollments.student_id
LEFT JOIN courses ON enrollments.course_id = courses.course_id;
"""

df = pd.read_sql_query(query, conn)
df


,name,title,grade
0,Lina,Databases,A
1,Lina,Python Programming,B
2,Omar,Python Programming,A
3,Omar,Statistics,B
4,Sara,Statistics,A
5,Sara,Machine Learning,C
6,Yousef,Databases,A
7,Yousef,Machine Learning,B
8,Khaled,None,None


Note:
INNER JOIN → the student "Khaled" will not appear because he is not enrolled in any course.

LEFT JOIN → his name will appear, and the title and grade columns will be NULL.



---
# Writing Subqueries (Nested SELECTs)


In [12]:
# Get the names of students who got an A in any course:
df = pd.read_sql_query('select name from students where student_id in (select student_id from enrollments where grade="A");',conn)
df

,name
0,Lina
1,Omar
2,Sara
3,Yousef


In [13]:
# Show course titles that "Lina" is enrolled in:
df = pd.read_sql_query('select title from courses where course_id in (select course_id from enrollments where student_id =(select student_id from students where name="Lina"));',conn)
df

,title
0,Databases
1,Python Programming


In [14]:
# Show students who are not enrolled in any course:
df = pd.read_sql_query('select name from students where student_id not in (select student_id from enrollments);',conn)
df

,name
0,Khaled
